In [1]:
# BUY RULE:
# if (bm > 0.75 AND mom12m > 0.65 AND mve < 0.15) OR (bm > 1.15 AND mom12m > -0.25):
#     signal = BUY

# SELL RULE:
# if (bm < -0.65 AND mom12m < -0.75) OR (mve < -1.0 AND mom12m < 0.25):
#     signal = SELL

# HOLD RULE:
# else:
#     signal = HOLD

In [6]:
import pandas as pd
import numpy as np

green_cleaned = pd.read_csv('../green cleaned.csv', dtype={'ncusip': 'string'})
green_cleaned['ret_fwd_1'] = (green_cleaned.groupby('permno')['ret_excess'].shift(-1) )

In [13]:
variables = ['datadate', 'ticker', 'comnam', 'bm', 'mom12m', 'mve', 'ret_fwd_1']

In [14]:
start_date = '2019-12-31'
end_date = '2024-11-30'

target_date = '2019-12-31'

# 3. Create the filtered DataFrame for your analysis
# analysis_df = green_cleaned[
#     (green_cleaned['datadate'] >= start_date) & 
#     (green_cleaned['datadate'] <= end_date)
# ][variables].copy()

analysis_df = green_cleaned[
    (green_cleaned['datadate'] == target_date)
][variables].copy()

conditions = [
    # BUY: bm > 0.75 AND mom12m > 0.35 AND mve > -0.5
    ( (analysis_df['bm'] > 0.75) & (analysis_df['mom12m'] > 0.65) & (analysis_df['mve'] < 0.15) ) | ((analysis_df['bm'] > 1.15) & (analysis_df['mom12m']>-0.25)),
    
    # SELL: bm < -0.5 AND mom12m < -0.75
    ( (analysis_df['bm'] < -0.65) & (analysis_df['mom12m'] < -0.75) ) | ( (analysis_df['mve'] < -1) & (analysis_df['mom12m'] < 0.25) )
]

# 3. Define choices
choices = ['BUY', 'SELL']

# 4. Create the 'signal' column
analysis_df['signal'] = np.select(conditions, choices, default='HOLD')

print("\n--- BUY ---")
print(analysis_df[analysis_df['signal'] == 'BUY'])

print("\n--- SELL ---")
print(analysis_df[analysis_df['signal'] == 'SELL'])


--- BUY ---
          datadate ticker                            comnam        bm  \
20084   2019-12-31    KMI                 KINDER MORGAN INC  1.778534   
27350   2019-12-31   NWSA                     NEWS CORP NEW  1.962701   
27449   2019-12-31   COTY                          COTY INC  1.374721   
32297   2019-12-31    SYF               SYNCHRONY FINANCIAL  1.480040   
37388   2019-12-31    HPE     HEWLETT PACKARD ENTERPRISE CO  1.810787   
88618   2019-12-31   ARNC                       ARCONIC INC  0.888443   
107122  2019-12-31    XRX               XEROX HOLDINGS CORP  2.192086   
109624  2019-12-31    HES                         HESS CORP  1.312411   
115387  2019-12-31   FITB               FIFTH THIRD BANCORP  1.816962   
163026  2019-12-31    LEN                       LENNAR CORP  2.037168   
166989  2019-12-31    PHM                   PULTE GROUP INC  0.863085   
179062  2019-12-31    BAC              BANK OF AMERICA CORP  1.937569   
182521  2019-12-31    PNC  P N C FINAN